In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp template.config

In [ ]:
# export
from typing import List
from fastcore.script import call_parse, Param
import inspect
import os
import importlib
import json
from pathlib import Path

from pymemri.plugin.pluginbase import get_plugin_cls
from pymemri.pod.client import PodClient

In [ ]:
# hide
# test imports
from pymemri.plugin.pluginbase import PluginBase
from pprint import pprint
import os

# Plugin Configuration

Often, plugins require some configuration before running. For example you might want to run plugin on data from a specific service, or run a specific version of your machine learning model. To configure your plugin when running it from the front-end, plugins require a `config.json` in the root of the plugin repository. This file contains a declarative definition, which our front-end app uses to display a configuration screen.

This module contains utilities to directly generate this config file from the plugin definition, by inferring all arguments from the plugin `__init__` method. All configuration fields are textboxes by default, which can be changed to different types in the future. For a full example, see our guide on [building plugins](https://memri.docs.memri.io/docs.memri.io/guides/build_and_deploy_your_model/).

Usage:
```
create_plugin_config
```

In [ ]:
# export
# hide
ALLOWED_TYPES = [int, str, float]

# export
def get_params(cls):
    params = inspect.signature(cls.__init__).parameters
    return {k: v for k, v in list(params.items()) if k not in {"self", "args", "kwargs"}}

def identifier_to_displayname(identifier: str) -> str:
    return identifier.replace("_", " ").title()

def get_param_config(name, dtype, is_optional, default):
    return {
        "name": name,
        "display": identifier_to_displayname(name),
        "data_type": dtype,
        "type": "textbox",
        "default": default,
        "optional": is_optional,
    }

In [ ]:
# export
def create_config(plugin_cls: type) -> List[dict]:
    """
    Returns a declarative plugin configuration, inferred from the `__init__` method signature of `plugin_cls`.
    This function is used internally by the `create_plugin_config` CLI. For general use, use the CLI instead.
    
    Arguments that start with `_`, untyped arguments or arguments that are not in `ALLOWED_TYPES` are skipped.

    Args:
        plugin_cls (type): A plugin class, inherited from PluginBase.

    Returns:
        List[dict]: A declarative configuration definition as list of dictionaries.
    """
    config = list()
    for param_name, param in get_params(plugin_cls).items():
        if param_name.startswith("_"):
            continue
        if param.annotation == inspect._empty:
            print(f"Skipping unannotated parameter `{param_name}`")
            continue
        if param.annotation not in ALLOWED_TYPES:
            print(f"Skipping parameter with unknown type: `{param_name}: {param.annotation}`")
            continue
        is_optional = param.default != inspect._empty
        dtype = PodClient.TYPE_TO_SCHEMA[param.annotation]
        default = param.default if is_optional else None
        param_config = get_param_config(param_name, dtype, is_optional, default)
        config.append(param_config)
    return config

In [ ]:
# export
@call_parse
def create_plugin_config(
    metadata: Param("metadata.json of the plugin", str) = "./metadata.json",
    tgt_file: Param("Filename of config file", str) = "config.json",
    schema_file: Param("Filename of exported plugin schema", str) = "schema.json"
):
    """
    Creates a plugin configuration definition from the arguments of your plugin class.
    
    Configuration arguments are inferred from the arguments of your plugin `__init__` method.
    Arguments that start with `_`, untyped arguments or arguments that are not in `ALLOWED_TYPES` are skipped.
    All generated fields are "textbox" by default, in the future our front-end will support more
    types of fields.
    Args:
        metadata (Param, optional): Location of the "metadata.json" file,
            Defaults to "./metadata.json"
        tgt_file (Param, optional): File the config definition is saved to.
            Defaults to "config.json".
    """
    if metadata is None:
        if os.path.exists("./metadata.json"):
            metadata = "./metadata.json"
        else:
            print("Define a metadata file with --metadata <filename>")
            return
    with open(metadata, "r") as f:
        metadata = json.load(f)
        
    plugin_module = metadata["pluginModule"]
    plugin_name = metadata["pluginName"]
    
    try:
        plugin_cls = get_plugin_cls(plugin_module, plugin_name)
    except Exception as e:
        print(e)
        return
    config = create_config(plugin_cls)
    
    with open(tgt_file, "w") as f:
        json.dump(config, f, indent=2)
    print(f"Config saved to {Path(tgt_file)}")
    
    plugin_schema = plugin_cls.get_schema()
    with open(config_file, "w") as f:
        json.dump(plugin_schema, f, indent=2)

### Example

As example, we create a test plugin with various configuration arguments. The `create_config` method is called on the plugin, and generates a declarative configuration field for each plugin argument. Note that arguments that start with a `_`, and untyped arguments are skipped in the annotation. 

In [ ]:
class MyPlugin(PluginBase):
    def __init__(
        self,
        my_arg: str,
        my_str: str = None,
        my_int: int = None,
        my_float: float = None,
        unannotated=None,
        _private=None,
        **kwargs
    ):
        super().__init__(**kwargs)

    def run(self):
        pass

    def add_to_schema(self):
        pass

In [ ]:
config = create_config(MyPlugin)
for c in config:
    print(f'{c["display"]} ({c["data_type"]})') 

Skipping unannotated parameter `unannotated`
My Arg (Text)
My Str (Text)
My Int (Integer)
My Float (Real)


In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted Untitled.ipynb.
Converted basic.ipynb.
Converted cvu.utils.ipynb.
Converted data.dataset.ipynb.
Converted data.photo.ipynb.
Converted exporters.exporters.ipynb.
Converted index.ipynb.
Converted itembase.ipynb.
Converted plugin.authenticators.credentials.ipynb.
Converted plugin.authenticators.oauth.ipynb.
Converted plugin.listeners.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.states.ipynb.
Converted plugins.authenticators.password.ipynb.
Converted pod.api.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.sync.ipynb.
Converted pod.utils.ipynb.
Converted template.config.ipynb.
Converted template.formatter.ipynb.
Converted test_schema.ipynb.
Converted test_utils.ipynb.
